<center><h1>Least Squares Support Vector Classifier</h1></center>

## Summary:
1. [Introduction](#introduction)


2. [Using the classifier](#using_classifier)

    2.1 [CPU/Numpy version](#cpu_version)
    
    2.2 [GPU/PyTorch version](#gpu_version)

# 1. Introduction <a class="anchor" id="introduction"></a>

The Least Squares Support Vector Machine (LSSVM) is a variation of the original Support Vector Machine (SVM) in which we have a slight change in the objective and restriction functions that results in a big simplification of the optimization problem.

First, let's see the optimization problem of an SVM:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + C \sum_{i=1}^{n} \xi_i &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b)\geq 1 - \xi_i, && i = 1,..., n \\
         && \xi_i \geq 0,                            && i = 1,..., n
\end{align}
$$

In this case, we have a set of inequality restrictions and when solving the optimization problem by it's dual we find a discriminative function, adding the kernel trick, of the type:


$$ f(\vec{x}) = sign \ \Big( \sum_{i=1}^{n} \alpha_i^o y_i K(\vec{x}_i,\vec{x}) + b_o \Big) $$

Where $\alpha_i^o$ and $b_o$ denote optimum values. Giving enough regularization (smaller values of $C$) we get a lot of $\alpha_i^o$ nulls, resulting in a sparse model in which we only need to save the pairs $(\vec{x}_i,y_i)$ which have the optimum dual variable not null. The vectors $\vec{x}_i$ with not null $\alpha_i^o$ are known as support vectors (SV).



In the LSSVM case, we change the inequality restrictions to equality restrictions. As the $\xi_i$ may be negative we square its values in the objective function:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + \gamma \frac{1}{2}\sum_{i=1}^{n} \xi_i^2 &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b) = 1 - \xi_i, && i = 1,..., n
\end{align}
$$


The dual of this optimization problem results in a system of linear equations, a set of Karush-Khun-Tucker (KKT) equations:

$$
\begin{bmatrix} 
    0 & \vec{d}^T \\
    \vec{y} & \Omega + \gamma^{-1} I 
\end{bmatrix}
\
\begin{bmatrix} 
    b  \\
    \vec{\alpha}
\end{bmatrix}
=
\begin{bmatrix} 
    0 \\
    \vec{1}
\end{bmatrix}
$$

Where, with the kernel trick, &nbsp; $\Omega_{i,j} = y_i y_j K(\vec{x}_i,\vec{x}_j)$,  &nbsp;  $\vec{y} = [y_1 \ y_2 \ ... \ y_n]^T$, &nbsp; $\vec{\alpha} = [\alpha_1 \ \alpha_2 \ ... \ \alpha_n]^T$ &nbsp;  e &nbsp; $\vec{1} = [1 \ 1 \ ... \ 1]^T$.

The discriminative function of the LSSVM has the same form of the SVM but the $\alpha_i^o$ aren't usually null, resulting in a bigger model. The big advantage of the LSSVM is in finding it's parameters, which is reduced to solving the linear system of the type:

$$ A\vec{z} = \vec{b} $$

A well-known solution of the linear system is when we minimize the square of the residues, that can be written as the optimization problem:

$$
\begin{align}
    minimize && f_o(\vec{z})=\frac{1}{2}||A\vec{z} - \vec{b}||^2\\
\end{align}
$$

And have the analytical solution:

$$ \vec{z} = A^{\dagger} \vec{b} $$

Where $A^{\dagger}$ is the pseudo-inverse defined as:

$$ A^{\dagger} = (A^T A)^{-1} A^T$$

# 2. Using the classifier <a class="anchor" id="using_classifier"></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler

from lssvm import LSSVC, LSSVC_GPU
from utils.encoding import dummie2multilabel

In [2]:
# Preprocessing

# Import digits recognition dataset (from sklearn)
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2020)

# Scaling features (from sklearn)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

X_train.shape: (1078, 64)
X_test.shape:  (719, 64)
y_train.shape: (1078,)
y_test.shape:  (719,)
np.unique(y_train): [0 1 2 3 4 5 6 7 8 9]
np.unique(y_test):  [0 1 2 3 4 5 6 7 8 9]


## 2.1 CPU/Numpy version <a class="anchor" id="cpu_version"></a>

In [3]:
# Use the classifier with different kernels

print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
print('acc_test = ', acc, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
print('acc_test = ', acc, '\n')

Gaussian kernel:
(1078, 1078)
1079
time =  0.2759546360030072


NameError: name 'manual_pinv' is not defined

If there are any questions about a specific method, the user can ask for help:

In [ ]:
help(LSSVC.fit)

The user can also have an overview of the class:

In [ ]:
help(LSSVC)

The user may also save the model in JSON format: 

In [ ]:
lssvc.dump('model')
loaded_model = LSSVC.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(loaded_model.predict(X_ts_norm))
    )
)

## 2.2 GPU/PyTorch version <a class="anchor" id="gpu_version"></a>    

It has the same functionalities and syntax of the CPU version, the difference is the use of PyTorch to run the operations on the GPU.

In [ ]:
# Use the classifier with different kernels

print('Gaussian kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='rbf', sigma=.5) 
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Polynomial kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='poly', d=2)
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Linear kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='linear')
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

The user may also save the model in JSON format: 

In [ ]:
lssvc_gpu.dump('model')
loaded_model = LSSVC_GPU.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(loaded_model.predict(X_ts_norm).cpu())
    )
)

The `model.json` is the same for the CPU and GPU version, giving the developer the possibility to train a model in GPU, dumping it in a .json, and loading in CPU version (the other way around is also possible).

In [ ]:
lssvc.dump('model_from_cpu')
lssvc_gpu = LSSVC_GPU.load('model_from_cpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)

In [ ]:
lssvc_gpu.dump('model_from_gpu')
lssvc = LSSVC.load('model_from_gpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)